In [1]:
import os
import glob
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import numpy as np
import random
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pprint
import pyspark
import pyspark.sql.functions as F

from pyspark.sql.functions import col
from pyspark.sql.types import StringType, IntegerType, FloatType, DateType

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, f1_score, roc_auc_score
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

import model_inference


In [2]:
# Build a .py script that takes a snapshot date, loads a model artefact and make an inference and save to datamart

## set up pyspark session

In [3]:
# Initialize SparkSession
spark = pyspark.sql.SparkSession.builder \
    .appName("dev") \
    .master("local[*]") \
    .getOrCreate()

# Set log level to ERROR to hide warnings
spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/14 04:54:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/14 04:54:24 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/05/14 04:54:24 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


## set up config

In [4]:
snapshot_date_str = "2024-01-01"
model_name = "credit_model_2024_09_01.pkl"


In [5]:
config = {}
config["snapshot_date_str"] = snapshot_date_str
config["snapshot_date"] = datetime.strptime(config["snapshot_date_str"], "%Y-%m-%d")
config["model_name"] = model_name
config["model_bank_directory"] = "model_bank/"
config["model_artefact_filepath"] = config["model_bank_directory"] + config["model_name"]

pprint.pprint(config)

{'model_artefact_filepath': 'model_bank/credit_model_2024_09_01.pkl',
 'model_bank_directory': 'model_bank/',
 'model_name': 'credit_model_2024_09_01.pkl',
 'snapshot_date': datetime.datetime(2024, 1, 1, 0, 0),
 'snapshot_date_str': '2024-01-01'}


## load model artefact from model bank

In [6]:
# Load the model from the pickle file
with open(config["model_artefact_filepath"], 'rb') as file:
    model_artefact = pickle.load(file)

print("Model loaded successfully! " + config["model_artefact_filepath"])

Model loaded successfully! model_bank/credit_model_2024_09_01.pkl


## load feature store

In [7]:
feature_location = "data/feature_clickstream.csv"

# Load CSV into DataFrame - connect to feature store
features_store_sdf = spark.read.csv(feature_location, header=True, inferSchema=True)
# print("row_count:",features_store_sdf.count())


# extract feature store
features_sdf = features_store_sdf.filter((col("snapshot_date") == config["snapshot_date"]))
print("extracted features_sdf", features_sdf.count(), config["snapshot_date"])

features_pdf = features_sdf.toPandas()
features_pdf

extracted features_sdf 8974 2024-01-01 00:00:00


,fe_1,fe_2,fe_3,fe_4,fe_5,fe_6,fe_7,fe_8,fe_9,fe_10,...,fe_13,fe_14,fe_15,fe_16,fe_17,fe_18,fe_19,fe_20,Customer_ID,snapshot_date
0,239,140,-24,265,2,-32,147,-38,280,210,...,235,140,45,179,61,38,68,171,CUS_0x1037,2024-01-01
1,-15,137,35,-9,124,23,-33,-66,134,118,...,305,52,30,214,-125,176,128,209,CUS_0x1069,2024-01-01
2,361,58,-37,150,17,40,189,-64,119,32,...,168,-32,-15,34,162,94,-55,103,CUS_0x114a,2024-01-01
3,96,231,-41,20,147,217,158,-45,8,300,...,249,-4,259,190,97,163,-40,3,CUS_0x1184,2024-01-01
4,54,294,-89,-43,288,355,57,-33,157,216,...,311,191,0,-19,-51,80,69,-26,CUS_0x1297,2024-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8969,141,154,89,149,155,114,-94,277,93,195,...,147,107,164,176,105,92,126,-19,CUS_0xdf6,2024-01-01
8970,96,52,242,123,207,155,147,189,121,291,...,60,218,132,104,-36,164,-45,158,CUS_0xe23,2024-01-01
8971,75,-7,-42,60,57,237,12,90,225,9,...,90,117,251,153,36,-117,244,3,CUS_0xe4e,2024-01-01
8972,138,153,38,306,282,171,-23,102,281,141,...,-75,43,64,182,210,-73,13,7,CUS_0xedd,2024-01-01


## preprocess data for modeling

In [8]:
# prepare X_inference
feature_cols = [fe_col for fe_col in features_pdf.columns if fe_col.startswith('fe_')]
X_inference = features_pdf[feature_cols]

# apply transformer - standard scaler
transformer_stdscaler = model_artefact["preprocessing_transformers"]["stdscaler"]
X_inference = transformer_stdscaler.transform(X_inference)

print('X_inference', X_inference.shape[0])
X_inference

X_inference 8974


array([[ 1.37343573,  0.38727197, -1.30102033, ..., -0.63976417,
        -0.3378844 ,  0.7008268 ],
       [-1.1576494 ,  0.35743607, -0.71008323, ...,  0.70437269,
         0.2513317 ,  1.06643414],
       [ 2.58915379, -0.4282426 , -1.43122681, ..., -0.09431733,
        -1.54577741,  0.04658209],
       ...,
       [-0.26080821, -1.07468707, -1.48130622, ..., -2.14948311,
         1.39048282, -0.91554248],
       [ 0.36698062,  0.51656086, -0.68003558, ..., -1.72091773,
        -0.87799916, -0.8770575 ],
       [ 1.2239622 , -0.3685708 , -0.77017853, ...,  0.85047452,
         0.4673776 ,  0.51802313]], shape=(8974, 20))

## model prediction inference

In [9]:
# load model
model = model_artefact["model"]

# predict model
y_inference = model.predict_proba(X_inference)[:, 1]

# prepare output
y_inference_pdf = features_pdf[["Customer_ID","snapshot_date"]].copy()
y_inference_pdf["model_name"] = config["model_name"]
y_inference_pdf["model_predictions"] = y_inference
y_inference_pdf

,Customer_ID,snapshot_date,model_name,model_predictions
0,CUS_0x1037,2024-01-01,credit_model_2024_09_01.pkl,0.296084
1,CUS_0x1069,2024-01-01,credit_model_2024_09_01.pkl,0.334873
2,CUS_0x114a,2024-01-01,credit_model_2024_09_01.pkl,0.350938
3,CUS_0x1184,2024-01-01,credit_model_2024_09_01.pkl,0.232311
4,CUS_0x1297,2024-01-01,credit_model_2024_09_01.pkl,0.191758
...,...,...,...,...
8969,CUS_0xdf6,2024-01-01,credit_model_2024_09_01.pkl,0.309572
8970,CUS_0xe23,2024-01-01,credit_model_2024_09_01.pkl,0.271457
8971,CUS_0xe4e,2024-01-01,credit_model_2024_09_01.pkl,0.171709
8972,CUS_0xedd,2024-01-01,credit_model_2024_09_01.pkl,0.385355


## save model inference to datamart gold table

In [10]:
# create bronze datalake
gold_directory = f"datamart/gold/model_predictions/{config["model_name"][:-4]}/"
print(gold_directory)

if not os.path.exists(gold_directory):
    os.makedirs(gold_directory)

# save gold table - IRL connect to database to write
partition_name = config["model_name"][:-4] + "_predictions_" + snapshot_date_str.replace('-','_') + '.parquet'
filepath = gold_directory + partition_name
spark.createDataFrame(y_inference_pdf).write.mode("overwrite").parquet(filepath)
# df.toPandas().to_parquet(filepath,
#           compression='gzip')
print('saved to:', filepath)

datamart/gold/model_predictions/credit_model_2024_09_01/
saved to: datamart/gold/model_predictions/credit_model_2024_09_01/credit_model_2024_09_01_predictions_2024_01_01.parquet


## backfill

In [11]:
# set up config
snapshot_date_str = "2023-01-01"

start_date_str = "2023-01-01"
end_date_str = "2024-12-01"

In [12]:
# generate list of dates to process
def generate_first_of_month_dates(start_date_str, end_date_str):
    # Convert the date strings to datetime objects
    start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
    end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
    
    # List to store the first of month dates
    first_of_month_dates = []

    # Start from the first of the month of the start_date
    current_date = datetime(start_date.year, start_date.month, 1)

    while current_date <= end_date:
        # Append the date in yyyy-mm-dd format
        first_of_month_dates.append(current_date.strftime("%Y-%m-%d"))
        
        # Move to the first of the next month
        if current_date.month == 12:
            current_date = datetime(current_date.year + 1, 1, 1)
        else:
            current_date = datetime(current_date.year, current_date.month + 1, 1)

    return first_of_month_dates

dates_str_lst = generate_first_of_month_dates(start_date_str, end_date_str)


In [13]:
for snapshot_date in dates_str_lst:
    print(snapshot_date)
    model_inference.main(snapshot_date, model_name)

2023-01-01


---starting job---


{'model_artefact_filepath': 'model_bank/credit_model_2024_09_01.pkl',
 'model_bank_directory': 'model_bank/',
 'model_name': 'credit_model_2024_09_01.pkl',
 'snapshot_date': datetime.datetime(2023, 1, 1, 0, 0),
 'snapshot_date_str': '2023-01-01'}
Model loaded successfully! model_bank/credit_model_2024_09_01.pkl
extracted features_sdf 8974 2023-01-01 00:00:00
X_inference 8974
datamart/gold/model_predictions/credit_model_2024_09_01/
saved to: datamart/gold/model_predictions/credit_model_2024_09_01/credit_model_2024_09_01_predictions_2023_01_01.parquet


---completed job---


2023-02-01


---starting job---


{'model_artefact_filepath': 'model_bank/credit_model_2024_09_01.pkl',
 'model_bank_directory': 'model_bank/',
 'model_name': 'credit_model_2024_09_01.pkl',
 'snapshot_date': datetime.datetime(2023, 2, 1, 0, 0),
 'snapshot_date_str': '2023-02-01'}
Model loaded successfully! model_bank/credit_model_2024_09_01.pkl
extracted features_sdf 8974 2023-02-01 

## Check datamart

In [14]:
# Initialize SparkSession
spark = pyspark.sql.SparkSession.builder \
    .appName("dev") \
    .master("local[*]") \
    .getOrCreate()

# Set log level to ERROR to hide warnings
spark.sparkContext.setLogLevel("ERROR")

In [15]:
folder_path = "datamart/gold/model_predictions/credit_model_2024_09_01/"
files_list = [folder_path+os.path.basename(f) for f in glob.glob(os.path.join(folder_path, '*'))]
df = spark.read.option("header", "true").parquet(*files_list)
print("row_count:",df.count())

df.show()

row_count: 215376
+-----------+-------------+--------------------+-------------------+
|Customer_ID|snapshot_date|          model_name|  model_predictions|
+-----------+-------------+--------------------+-------------------+
| CUS_0x2ff7|   2023-09-01|credit_model_2024...|0.30120009183883667|
| CUS_0x303a|   2023-09-01|credit_model_2024...|0.28625059127807617|
| CUS_0x305c|   2023-09-01|credit_model_2024...| 0.3428035378456116|
| CUS_0x3082|   2023-09-01|credit_model_2024...|0.28813859820365906|
| CUS_0x308d|   2023-09-01|credit_model_2024...| 0.3139890134334564|
| CUS_0x3101|   2023-09-01|credit_model_2024...|0.22602568566799164|
| CUS_0x3127|   2023-09-01|credit_model_2024...| 0.2247123271226883|
| CUS_0x3161|   2023-09-01|credit_model_2024...| 0.2053392231464386|
| CUS_0x3187|   2023-09-01|credit_model_2024...|0.19738353788852692|
| CUS_0x31b8|   2023-09-01|credit_model_2024...|0.39255911111831665|
| CUS_0x31c0|   2023-09-01|credit_model_2024...|0.36157599091529846|
| CUS_0x3214|   